# Multiconfigurational linear response

## Concept

So-far we have seen CI response and how pure CASCI response can fail dramatically due to the neglect of orbital relaxation. We fixed it in a simple way using state-averaging, but this also introduced some difficulties, such as the dependency of the result on the number of states and the need to include all orbitals involved in the excitation in the active space.

These difficulties can be avoided altogether by deriving a proper linear response for MCSCF. For this, we introduce our parameterization

$$
  | \bar{\psi}(t) \rangle = e^{-i\hat{\kappa}(t)} e^{-i\hat{R}(t)} |0\rangle 
$$

which contains both the orbital rotation operator similar to HF or DFT response

$$
  \hat{\kappa}(t) =
  \sum_a^\mathrm{act,unocc} \sum_i^\mathrm{occ,act} \left[
    \kappa_{ai}(t) \hat{a}^\dagger_a \hat{a}_i + \kappa_{ai}^*(t) \hat{a}^\dagger_i \hat{a}_a
    \right]
$$

but also the CI state-transfer operator

$$
  \hat{R}(t) = \sum_n R_{n0}(t) | n \rangle \langle 0 | + R^*_{0n}(t) | 0 \rangle \langle n | $$

For CASSCF wave functions, there is a redundancy between the active-active orbital response parameters and the CI response parameters, so these orbital response terms are dropped. The orbital rotations are thus restricted to inactive-active, inactive-virtual and active-virtual.

This theory can thus be seen as CASCI response but with added orbital excitations to and from non-active orbitals. Hence, it fixes the lack of orbital relaxation of CASCI and also lifts the need to include all orbitals involved in the excitations within our active space. The only difference between having the orbitals inside or outside the active space is the description of the reference state (typically the ground state) and the fact that the CI state-transfer includes all order of excitations while the orbital operators are restricted to single excitations.

## Illustrative calculations

Let us go back to our water example.

In [1]:
import multipsi as mtp
import veloxchem as vlx

* Warning * Environment variable OMP_NUM_THREADS not set.
* Warning * Setting OMP_NUM_THREADS to 8.


In [2]:
mol_str = """3

O       0.0000000000     0.0000000000     0.1178336003
H      -0.7595754146    -0.0000000000    -0.4713344012
H       0.7595754146     0.0000000000    -0.4713344012
"""

molecule = vlx.Molecule.read_xyz_string(mol_str)
basis = vlx.MolecularBasis.read(molecule, "6-31g")
scf_drv = vlx.ScfRestrictedDriver()
scf_results = scf_drv.compute(molecule, basis)

* Info * Reading basis set from file: /opt/anaconda3/envs/echem/lib/python3.11/site-packages/veloxchem/basis/6-31G        
                                                                                                                          
                                              Molecular Basis (Atomic Basis)                                              
                                                                                                                          
                               Basis: 6-31G                                                                               
                                                                                                                          
                               Atom Contracted GTOs           Primitive GTOs                                              
                                                                                                                          
                

In [3]:
import py3Dmol as p3d
viewer = p3d.view(width=300, height=200)
viewer.addModel(mol_str, 'xyz')
viewer.setViewStyle({"style": "outline", "width": 0.05})
viewer.setStyle({"stick":{},"sphere": {"scale":0.25}})
viewer.rotate(-90,'x')
viewer.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [4]:
# CAS(6,5) calculation of water
space = mtp.OrbSpace(molecule, scf_drv.mol_orbs)
space.cas(6, 5)  # 3 O_2p and 2 H_1s

# CASSCF calculation
mcscf_drv = mtp.McscfDriver()
mcscf_results = mcscf_drv.compute(molecule, basis, space)

                                                                                                                          
                          Multi-Configurational Self-Consistent Field Driver
                                                                                                                          
        ╭────────────────────────────────────╮
        │          Driver settings           │
        ╰────────────────────────────────────╯
          State-specific calculation
          Max. iterations         : 50
          BFGS window             : 5
          Convergence thresholds:
            - Energy change       : 1e-08
            - Gradient norm       : 0.0001
                                                                                                                          
          Integrals in memory
                                                                                                                          

          Active space def

In [5]:
mcrpa = mtp.Mclr_EigenSolver()
# We only need 4 states but use 5 states to help a bit with convergence
rsp_dict = mcrpa.compute(molecule, basis, mcscf_drv, n_states=5)

Warning, significant negative value in orbital S(2): -0.37206280256720925
                                                                                                                          
Initialization time: 00:00:00
                                                                                                                          
          Linear eigensolver
          ------------------
                                                                                                                          
CI parameters:      54
Orbital parameters: 52
                                                                                                                          
                                                                                                                          
                                                                                                                          
        MC-RPA Iterations
        -----------------
  

In [6]:
# Results computed in CI response and SA-MCSCF
FCI_excitation_energies = [8.44992, 10.68451, 10.97654, 13.37692]
CIS_excitation_energies = [9.38059, 11.30739, 11.83172, 13.90237]
CASCI_excitation_energies = [18.61974291, 18.73087038, 26.61579239, 28.51654574]
SA_CASSCF_excitation_energies = [7.55787583, 9.83566209, 10.32868402, 12.56323891]
au2ev = 27.211386

# Multiconfigurational linear response for CAS(6,5)
MCRPA_6_5_excitation_energies = au2ev * rsp_dict["eigenvalues"][:4]

print("Full CI Energies")
print(FCI_excitation_energies)
print("CIS Energies")
print(CIS_excitation_energies)
print("SA-CASSCF Energies")
print(SA_CASSCF_excitation_energies)
print("CASCI Energies")
print(CASCI_excitation_energies)
print("MC-RPA Energies")
print(MCRPA_6_5_excitation_energies)

Full CI Energies
[8.44992, 10.68451, 10.97654, 13.37692]
CIS Energies
[9.38059, 11.30739, 11.83172, 13.90237]
SA-CASSCF Energies
[7.55787583, 9.83566209, 10.32868402, 12.56323891]
CASCI Energies
[18.61974291, 18.73087038, 26.61579239, 28.51654574]
MC-RPA Energies
[ 9.00809375 11.13046476 11.73264413 13.91373449]


We can see that this multiconfigurational response results is significantly better than CASCI, and is actually even better than SA-CASSCF and CIS.

It is also useful to look at the output of the calculation to understand what is happening. In particular, the code prints the number of trial vectors that are used to converge the calculation. We can see that MC-RPA uses both CI vectors and orbital vectors, corresponding to the 2 parts of our parametrization. The final response vector is a linear combination of both. We can also print the excitation character analysis to see this:

In [7]:
mcrpa.get_excitation_character()

                                                                                                                          
    Excitation characters
    ---------------------
                                                                                                                          
* State 1
- Exc. energy: 0.3310413425168189
 Electrons     Excitation
 0.391    :    3  ->    8
 0.574    :    3  ->    7
 CI weight: 0.616  Orb weight: 0.384
                                                                                                                          
* State 2
- Exc. energy: 0.40903703907882255
 Electrons     Excitation
 0.275    :    3  ->    9
 0.664    :    3  ->    6
 CI weight: 0.700  Orb weight: 0.300
                                                                                                                          
* State 3
- Exc. energy: 0.43116672307762466
 Electrons     Excitation
 0.279    :    4  ->    8
 0.207    :    2  ->    7
 0.158    :   

The orbital analysis shows the weights of the CI and orbital contribution to each response vector. Even if all of those states correspond broadly to excitations between valence orbitals which are all in the active space, all contain a sizeable contribution to the orbital response, which explains why they were so poorly described by CASCI.

We can go even further and drop the oxygen lone pair from our active space. As seen in the occupation number of the CASSCF calculation, the first orbital (the lone pair) is almost perfectly doubly occupied, and was only added in our active space because it is the HOMO of our system and is thus excited in the lowest excited states. Since we now also include excitations outside of the active space, we can safely remove this orbital and focus on the 2 pairs of O-H $\sigma$ and $\sigma^*$ (4 electrons in 4 orbitals):

In [8]:
space.cas(4, 4)  # Remove the first orbital
# CASSCF calculation
mcscf_drv = mtp.McscfDriver()
mcscf_results = mcscf_drv.compute(molecule, basis, space)

                                                                                                                          
                          Multi-Configurational Self-Consistent Field Driver
                                                                                                                          
        ╭────────────────────────────────────╮
        │          Driver settings           │
        ╰────────────────────────────────────╯
          State-specific calculation
          Max. iterations         : 50
          BFGS window             : 5
          Convergence thresholds:
            - Energy change       : 1e-08
            - Gradient norm       : 0.0001
                                                                                                                          
          Integrals in memory
                                                                                                                          

          Active space def

In [10]:
mcrpa = mtp.Mclr_EigenSolver()
rsp_dict = mcrpa.compute(molecule, basis, mcscf_drv, n_states=4)

                                                                                                                          
Initialization time: 00:00:00
                                                                                                                          
          Linear eigensolver
          ------------------
                                                                                                                          
CI parameters:      20
Orbital parameters: 54
                                                                                                                          
                                                                                                                          
                                                                                                                          
        MC-RPA Iterations
        -----------------
                                                                            

In [11]:
MCRPA_4_4_excitation_energies = au2ev * rsp_dict["eigenvalues"]

print("Full CI Energies")
print(FCI_excitation_energies)
print("CIS Energies")
print(CIS_excitation_energies)
print("SA-CASSCF Energies")
print(SA_CASSCF_excitation_energies)
print("MC-RPA Energies CAS(6,5)")
print(MCRPA_6_5_excitation_energies)
print("MC-RPA Energies CAS(4,4)")
print(MCRPA_4_4_excitation_energies)

Full CI Energies
[8.44992, 10.68451, 10.97654, 13.37692]
CIS Energies
[9.38059, 11.30739, 11.83172, 13.90237]
SA-CASSCF Energies
[7.55787583, 9.83566209, 10.32868402, 12.56323891]
MC-RPA Energies CAS(6,5)
[ 9.00809375 11.13046476 11.73264413 13.91373449]
MC-RPA Energies CAS(4,4)
[ 8.99461578 11.11512359 11.72113816 13.90719454]


As expected, since this orbital was almost perfectly doubly occupied, removing it from the active space made almost no difference in the excitation energies (this would not be true in SA-CASSCF).

As a result, the MC-RPA calculation used fewer CI trial vectors, and we can also see it in the excitation character analysis where the first 2 states are now purely described by the orbital response parameters:

In [12]:
mcrpa.get_excitation_character()

                                                                                                                          
    Excitation characters
    ---------------------
                                                                                                                          
* State 1
- Exc. energy: 0.3305460361595478
 Electrons     Excitation
 0.578    :    3  ->    7
 0.393    :    3  ->    8
 CI weight: 0.000  Orb weight: 1.000
                                                                                                                          
* State 2
- Exc. energy: 0.40847326146200613
 Electrons     Excitation
 0.670    :    3  ->    6
 0.275    :    3  ->    9
 CI weight: 0.000  Orb weight: 1.000
                                                                                                                          
* State 3
- Exc. energy: 0.4307438863711089
 Electrons     Excitation
 0.282    :    4  ->    8
 0.205    :    2  ->    7
 0.157    :    

Finally, as for HF or DFT response, we can invoke the Tamm-Dancoff approximation (TDA), which typically gives rather moderate error compared to the full RPA response although leads to only modest gains in computational time:

In [14]:
mcrpa = mtp.Mclr_EigenSolver()
mcrpa.TDA = True
rsp_dict = mcrpa.compute(molecule, basis, mcscf_drv, n_states=4)

                                                                                                                          
Initialization time: 00:00:00
                                                                                                                          
          Linear eigensolver
          ------------------
                                                                                                                          
CI parameters:      20
Orbital parameters: 54
                                                                                                                          
                                                                                                                          
                                                                                                                          
        MC-RPA Iterations
        -----------------
                                                                            

In [15]:
MCTDA_4_4_excitation_energies = au2ev * rsp_dict["eigenvalues"]

print("Full CI Energies")
print(FCI_excitation_energies)
print("MC-RPA Energies CAS(4,4)")
print(MCRPA_4_4_excitation_energies)
print("MC-TDA Energies CAS(4,4)")
print(MCTDA_4_4_excitation_energies)

Full CI Energies
[8.44992, 10.68451, 10.97654, 13.37692]
MC-RPA Energies CAS(4,4)
[ 8.99461578 11.11512359 11.72113816 13.90719454]
MC-TDA Energies CAS(4,4)
[ 8.99461578 11.11512359 11.72113816 13.90719454]
